In [9]:
!pip install pyarrow
!pip install fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.9 MB/s eta 0:00:00


In [10]:
import pandas as pd

In [13]:
january = pd.read_parquet('yellow_tripdata_2023-01.parquet', engine='fastparquet')
february = pd.read_parquet('yellow_tripdata_2023-02.parquet', engine='fastparquet')

In [18]:
len(january.columns)

19

In [20]:
january.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [25]:
january.sample(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1433079,2,2023-01-16 12:54:18,2023-01-16 13:12:18,4.0,3.16,1.0,N,234,142,4,19.1,0.0,0.5,0.00,0.00,1.0,23.10,2.5,0.00
2712233,2,2023-01-28 21:48:50,2023-01-28 22:00:27,2.0,1.38,1.0,N,230,233,1,11.4,1.0,0.5,3.28,0.00,1.0,19.68,2.5,0.00
657336,2,2023-01-08 14:32:32,2023-01-08 15:19:12,4.0,18.17,2.0,N,132,230,2,70.0,0.0,0.5,0.00,6.55,1.0,81.80,2.5,1.25
1642268,1,2023-01-18 16:59:28,2023-01-18 17:07:19,1.0,1.30,1.0,N,144,107,1,9.3,5.0,0.5,3.16,0.00,1.0,18.96,2.5,0.00
2695070,1,2023-01-28 18:35:04,2023-01-28 18:35:37,1.0,0.00,5.0,N,151,151,1,70.0,0.0,0.0,17.75,0.00,1.0,88.75,0.0,0.00


In [30]:
january['duration'] = (january.tpep_dropoff_datetime - january.tpep_pickup_datetime).dt.total_seconds() / 60

In [35]:
january.duration.std()

np.float64(42.59435124195458)

In [38]:
january.duration.between(1,60).sum()

np.int64(3009173)

In [44]:
january.duration.count()

np.int64(3066766)

In [46]:
january.duration.between(1,60).sum() / january.duration.count() * 100

np.float64(98.1220282212598)